# Warehouse Location Example
This notebook solves a simple warehouse location problem.

## Library and Data Preparation

### DOcplex Python API Installation
First we install the docplex python API which we will use for modeling the problem and solving it.

In [1]:
!pip install --user docplex -q

### Data import from csv files and normalizations

Importing the csv file with the problem parameters

In [3]:
import os, pandas as pd, numpy as np
data_parameters = pd.read_csv('Parameters.csv',index_col='ParameterName')
data_parameters.head()

,Value
ParameterName,
NbStores,10
Fixed,30


Import the csv file with the warehouse capacities

In [4]:
data_capacity = pd.read_csv('Capacity.csv',index_col='Warehouse')
warehouses = data_capacity.index
capacities = data_capacity
capacities.head()

,Capacity
Warehouse,
Bonn,1
Bordeaux,4
London,2
Paris,1
Rome,3


Import the csv file with the distances from warehouses to stores

In [5]:
data_distances = pd.read_csv('Distances.csv')
data_distances.head()

,Bonn,Bordeaux,London,Paris,Rome
0,20,24,11,25,30
1,28,27,82,83,74
2,74,97,71,96,70
3,2,55,73,69,61
4,46,96,59,83,4


Normalize distances table from 2D to a standard from-to table

In [6]:
stores = data_distances.index
warehouse_store_index = pd.Index([(w,s) for w in warehouses for s in stores])
distances = pd.Series( (data_distances.loc[s,w] for (w,s) in warehouse_store_index), index=warehouse_store_index)
distances.head()

Bonn  0    20
      1    28
      2    74
      3     2
      4    46
dtype: int64

## Optimization model


### Setup

In [7]:
# Library import for CPLEX Python API
from docplex.mp.model import Model

# Model instantion and naming
mdl = Model(name="WarehouseTest")

### Decision Variables

In [8]:
# Open Warehouse
xOpenWarehouse = mdl.binary_var_dict(keys=warehouses,lb=0,ub=1,name="Open")

In [9]:
# Supply of store from warehoues
xSupplyFlow = mdl.binary_var_dict(keys=warehouse_store_index,lb=0,ub=1,name="Supply")

### Objective Function

In [10]:
# Fixed cost of opening a warehouse
objOpeningCost = mdl.sum(data_parameters.loc['Fixed','Value'] * xOpenWarehouse[w] for w in warehouses)
# Transportation cost from each warehouse to each store
objSupplyCost = mdl.sum(distances[ws] * xSupplyFlow[ws] for ws in warehouse_store_index)

mdl.minimize(objOpeningCost + objSupplyCost)

### Constraints

In [11]:
# Pick exactly one warehouse to supply each store
for s in stores:
    mdl.add_constraint((mdl.sum(xSupplyFlow[ws] for ws in warehouse_store_index if ws[1]==s) == 1),ctname="ctPickWarehouse")

In [12]:
# Only use open warehouses to supply stores
for ws in warehouse_store_index:
    for w in warehouses:
        if w == ws[0]:
            mdl.add_constraint(xSupplyFlow[ws] <= xOpenWarehouse[w],ctname="ctOnlyOne")

In [13]:
# Respect capacity for each warehouse
for w in warehouses:
    mdl.add_constraint(mdl.sum(xSupplyFlow[ws] for ws in warehouse_store_index if ws[0] == w) <= capacities.loc[w,'Capacity'],ctname="ctCapacity")

## Solve the model

In [14]:
# Print model information
mdl.print_information()

# Solve the model
assert mdl.solve(), "!!! Solve of the model fails"

Model: WarehouseTest
 - number of variables: 55
   - binary=55, integer=0, continuous=0
 - number of constraints: 65
   - linear=65
 - parameters: defaults


DOcplexException: CPLEX runtime not found: please install CPLEX or solve this model on DOcplexcloud

In [149]:
# Print the solution
mdl.print_solution()

objective: 383
  "Open_Bonn"=1
  "Open_Bordeaux"=1
  "Open_London"=1
  "Open_Rome"=1
  "Supply_('London', 7)"=1
  "Supply_('Bonn', 3)"=1
  "Supply_('London', 9)"=1
  "Supply_('Rome', 0)"=1
  "Supply_('Bordeaux', 1)"=1
  "Supply_('Rome', 4)"=1
  "Supply_('Bordeaux', 5)"=1
  "Supply_('Bordeaux', 6)"=1
  "Supply_('Bordeaux', 8)"=1
  "Supply_('Rome', 2)"=1


In [150]:
#mdl.export_as_lp("warehouse.lp")